# 传统特征构建方法

重点：特征构建

传统方法：词袋模型，one-hot编码，TF-IDF

词袋模型的缺陷：
1. 数据稀疏
2. 维度巨大
3. 没有考虑上下文信息

![onehot](figures/onehot.png)

# Word2Vec介绍

Word2vec 是 [Mikolov et al.](http://arxiv.org/pdf/1301.3781.pdf) 提出一种训练词向量的方法。思路是通过训练，将每个词都映射到一个较短的词向量上来。所有的这些词向量就构成了向量空间，进而可以用数学的方法来研究词与词之间的关系。比如下图我们将词汇表里的词用"Royalty","Masculinity", "Femininity"和"Age"4个维度来表示，King这个词对应的词向量可能是(0.99,0.99,0.05,0.7)。当然在实际情况中，我们并不能对词向量的每个维度做一个很好的解释。

![](figures/word2vec/wordvector.png)

有了词向量，我们就可以分析词之间的关系，比如我们将词的维度降维到2维，有一个有趣的研究表明，用下图的词向量表示我们的词时，我们可以发现：
$$\vec {King} - \vec {Man} + \vec {Woman} = \vec {Queen}$$

![](figures/word2vec/goodexample.png)

Word2vec有 Continuous Bag-of-Words model (CBOW) and the Skip-Gram model 两种训练方式，前者是用一个词序列窗口中的其他词来预测中心词，后者则是用中心词来预测其他词。在实际使用时，一般使用 Skip-Gram 结合 [Negative Sampling](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) 进行训练。

![CBOW & skip-gram](figures/word2vec.png)

## Huffman Tree（霍夫曼树）
哈夫曼树是一种带权路径长度最短的二叉树，也称为最优二叉树。

有A B C D 四个词，数字表示词频，构造过程如下：

![softmax](figures/huffman1.jpg)

## Softmax

![softmax](figures/softmax.png)

## Hierarchical Softmax

在word2vec中，从隐藏层到输出的softmax,为了避免要计算所有词的softmax概率，word2vec采样了霍夫曼树来代替从隐藏层到输出softmax层的映射。
采用了二元逻辑回归的方法，即规定沿着左子树走，那么就是负类(霍夫曼树编码1)，沿着右子树走，那么就是正类(霍夫曼树编码0)。判别正类和负类的方法是使用sigmoid函数，即：

$$P(+) = \sigma(x_w^T\theta) = \frac{1}{1+e^{-x_w^T\theta}}$$

其中$x_w$是当前内部节点的词向量，而θ则是我们需要从训练样本求出的逻辑回归的模型参数。

![softmax](figures/CBOW.jpg)

![softmax](figures/hierarchicalsoftmax.jpg)

使用最大似然法来寻找所有节点的词向量和所有内部节点$\theta$。以上面的$w_2$例子来看，我们期望最大化下面的似然函数：

$$\prod_{i=1}^3P(n(w_i),i) = (1- \frac{1}{1+e^{-x_w^T\theta_1}})(1- \frac{1}{1+e^{-x_w^T\theta_2}})\frac{1}{1+e^{-x_w^T\theta_3}}$$

对于所有的训练样本，我们期望最大化所有样本的似然函数乘积。实际计算时为了减少梯度计算量，并没有把所有样本的似然乘起来得到真正的训练集最大似然，每次只用一个样本更新梯度。

In [5]:
# gensim 库中包含了 word2vec 模块
from gensim.models import word2vec

D:\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
sentences = word2vec.LineSentence('./in_the_name_of_people_segment.txt')
model = word2vec.Word2Vec(sentences, hs=1,min_count=1,window=3,vector_size=200) # hs:是否使用Hierarchical Softmax

# 保存模型
model.save("word2vec.model")

In [5]:
# 加载模型
fname = "word2vec.model"
model = word2vec.Word2Vec.load(fname)

In [6]:
# 获取相似词
word = "高育良"
model.wv.similar_by_word(word, topn =10)

[('沙瑞金', 0.9531140327453613),
 ('侯亮平', 0.9398585557937622),
 ('李达康', 0.9386978149414062),
 ('祁同伟', 0.929216742515564),
 ('陆亦可', 0.9156097173690796),
 ('季昌明', 0.9148707985877991),
 ('开玩笑', 0.9083293676376343),
 ('咱', 0.9017901420593262),
 ('叹', 0.894492506980896),
 ('牙疼', 0.8940712213516235)]

In [7]:
# 获取词向量
model.wv['京州']

array([-0.01535346,  0.3604732 ,  0.03981665,  0.18880066,  0.6338018 ,
        0.28114927,  0.05964074,  0.25281897, -0.4284452 ,  0.26713574,
        0.03453567, -0.02314873, -0.11728395,  0.0027645 ,  0.03520486,
       -0.20720787,  0.03071065,  0.42624998, -0.5083926 , -0.7377485 ,
        0.5243193 ,  0.20968246, -0.28458962, -0.07346202, -0.08507089,
       -0.21286307, -0.07201812,  0.23646404, -0.6476278 ,  0.29151472,
        0.35351804, -0.35985184, -0.44982687, -0.01178524, -0.19293083,
        0.16130134,  0.36793253,  0.10603456,  0.16723225, -0.50945866,
       -0.2526467 ,  0.09486602, -0.01863177,  0.12769422,  0.36140692,
        0.22467852, -0.00612385,  0.10622723,  0.10884923,  0.36551276,
        0.4004716 ,  0.17523465, -0.06098179, -0.3204234 , -0.04869512,
       -0.11654644, -0.07157538, -0.5358171 , -0.4994673 ,  0.01718854,
        0.06543393,  0.10025836, -0.01308374, -0.21079598, -0.3025868 ,
        0.22448917,  0.53149885,  0.2600528 , -0.17486493,  0.31

# 利用Word2Vec结合传统机器学习进行情感分析

In [1]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')
# df = df.head(5000)

In [3]:
train_corpus = [line.split() for line in df['review']]

In [6]:
model = word2vec.Word2Vec(
    train_corpus,
    vector_size=50,  # 词向量的维度
    workers=4)  # 调用的进程数
# 常用Word2Vec的参数：min_count,如果词频小于 min_count, word2vec 不会把这个词放入 vocab 里

# 保存模型
model.save('imdb.d2v')

利用训练好的词向量来进行情感分析

In [7]:
# 加载模型
model = word2vec.Word2Vec.load('imdb.d2v')

In [8]:
from sklearn.model_selection import train_test_split

X = df['review'].values
y = df['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# 用 word vec 的均值作为 doc vec
def get_doc_vec(sentence, model):
    scores = np.array([model.wv[word] for word in sentence.split() if word in model.wv])
    return np.mean(scores, axis=0)

In [10]:
import numpy as np
X_word2vec_train = np.array([get_doc_vec(sentence, model) for sentence in X_train])
X_word2vec_test =  np.array([get_doc_vec(sentence, model) for sentence in X_test])

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_word2vec_train,y_train)

C:\Users\陈继延\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [12]:
y_test_pred = lr.predict(X_word2vec_test)

In [13]:
acc = lr.score(X_word2vec_test,y_test)
acc

0.8067